# Data formatting  

All files can be found in [this Drive](https://drive.google.com/drive/folders/19jE98GeXH6Ahxp4WmupmGIR6h4r8yjEi?usp=sharing). The scripts and documentation files can be found in [this gitgub](https://github.com/dfortizgu/Google-Capstone-Project.git)

Reviewing the original files for the data we find some problems in the CSV files that contain de Infringements (Comparendos_20XX_Bogota.csv). Some of the files have format problems, having line jumps in places where should not be, generating problems for reading this in excel as csv files. For making this process automatic (since we have thousands of this problems in each file) we are going to read each file and generate a new one with the proper format.

In [2]:
import csv
import re

files_names = ["original_files/Comparendos_2015_Bogota.csv", "original_files/Comparendos_2016_Bogota.csv", "original_files/Comparendos_2017_Bogota.csv",
               "original_files/Comparendos_2018_Bogota.csv", "original_files/Comparendos_2019_Bogota.csv", "original_files/Comparendos_2020_Bogota.csv", "original_files/Comparendos_2021_Bogota.csv"]


#The next code corrects the wrong line jumps inside the csv files.

# Regex patter that allows to not consider the commas inside double quotes (" ") as separators. Regex pattern extracted from https://stackoverflow.com/questions/2785755/how-to-split-but-ignore-separators-in-quoted-strings-in-python

PATTERN = re.compile(r'''((?:[^,"']|"[^"]*"|'[^']*')+)''')


#The loop goes file by file searching for the lines with mistakes and correcting them, writing in the "new_Comparendos_20XX_Bogota.csv" files the new corrected lines (and the ones that were good)

total_corrections = 0

for name in files_names:

    file = open(name, encoding="utf8")
    new_file = open("new_"+name.replace("original_files/", ""), "w",encoding="utf8", newline="")
    write = csv.writer(new_file)


    # Remove the "\n" strings which appears explicitly in the final files as we ar using the csv.write to write the new files
    columns = file.readline().split(",")
    columns[-1] = columns[-1].replace("\n","")

    # Take the headers as the max amount of columns the file contains and write them in the new files
    num_columns = len(columns)
    write.writerow(columns)

    #For register lets see the non resolved mistakes (errors) and the ones resolved (correction). All corrections add up to total_corrections variable
    errors = 0
    corrections = 0

    for line in file:
        #Split bt the regex pattern 
        cells = PATTERN.split(line)[1::2]

        #Write the lines with the proper amount of columns
        if len(cells) == num_columns:
            cells[-1] = cells[-1].replace("\n","")
            write.writerow(cells)
            continue
        

        aux = False
        # When finding a line with the wrong number of columns, the next line is added until getting the right ampunt of columns for the line
        while len(cells) != num_columns:
            line = line.replace("\n","")
            line = line + file.readline()
            cells = PATTERN.split(line)[1::2]
            cells[-1] = cells[-1].replace("\n","")
            if len(cells) == num_columns:
                write.writerow(cells)
                aux = True
                break
        
        # Once the correction is made, we add to corrections and jump to the nex line to repeat the process
        if aux: 
            corrections += 1
            continue
        #If the line does not contain the correct number of columns a non corrected error is added
        errors += 1

    total_corrections += corrections

    print("File:", name)      
    print("Lines with errors not corrected: ",errors)
    print("Corrections: ",corrections)
    print("")

    file.close()

print("Total corrections made: ", total_corrections)

File: original_files/Comparendos_2015_Bogota.csv
Lines with errors not corrected:  0
Corrections:  7368

File: original_files/Comparendos_2016_Bogota.csv
Lines with errors not corrected:  0
Corrections:  14305

File: original_files/Comparendos_2017_Bogota.csv
Lines with errors not corrected:  0
Corrections:  20752

File: original_files/Comparendos_2018_Bogota.csv
Lines with errors not corrected:  0
Corrections:  26489

File: original_files/Comparendos_2019_Bogota.csv
Lines with errors not corrected:  0
Corrections:  17873

File: original_files/Comparendos_2020_Bogota.csv
Lines with errors not corrected:  0
Corrections:  0

File: original_files/Comparendos_2021_Bogota.csv
Lines with errors not corrected:  0
Corrections:  27973

Total corrections made:  114760


For applying the previous code the only file modified manually was "Comparendos_2019_Bogota.csv". This file had two errors that were easier to correct this way. 
1. The first one was a missing value in the LOCALIDAD (location) field for the row with OBJECTID = 20085078. It was filled with "2-CHAPINERO", the location corresponding to the latitude and longitude in that row.

2. The Second correction was replacing the text

        'CONDUCIR EN ESTADO DE EMBRIAGUEZ O BAJO LOS EFECTOS DE SUSTANCIAS ALUCINÓGENAS, SE ATENDERÁ A LO ESTABLECIDO EN EL ARTÍCULO 152 DE ESTE CÓDIGO. SI SE TRATA DE CONDUCTORES DE VEHÍCULOS DE SERVICIO PÚBLICO, DE TRANSPORTE ESCOLAR O DE INSTRUCTOR DE CONDUCCIÓN, LA MULTA PECUNIARIA Y EL PERÍODO DE SUSPENSIÓN DE LA LICENCIA SE DUPLICARÁN.
        "'

    with
    
        '"'
    
    since this was a repeated line for which the code was not working and implementing a particular solution was harder and more time consuming than just press "replace all" in the text editor.



